# 필요한 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import MySQLdb

# SQL DB 호출 및 활용

In [3]:
db = MySQLdb.connect(host='localhost', port=3306, user='root', password='1111', charset='utf8')
cursor = db.cursor()

In [5]:
sql = 'show databases;'
cursor.execute(sql)
cursor.fetchall()

(('econ_news_db',),
 ('information_schema',),
 ('megacoffeedb',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('stockdb',),
 ('sys',),
 ('world',))

In [ ]:
sql = 'create database if not exists econ_news_db;'
cursor.execute(sql)

In [6]:
# 생성 db 사용
sql = 'use econ_news_db;'
cursor.execute(sql)

0

In [9]:
# 생성 db 내 테이블 생성
sql = '''
create table if not exists news_table(
제목 text,
작성시간 text,
본문 text,
분류 text
);
'''
cursor.execute(sql)

0

In [7]:
# 테이블 생성 후 내용 확인
sql = 'show tables;'
cursor.execute(sql)
cursor.fetchall()

(('news_table',),)

In [21]:
# db 반영 데이터 개수 확인
sql = 'SELECT COUNT(제목) FROM news_table;'
cursor.execute(sql)
cursor.fetchall()

((2071062,),)

In [9]:
# 특수기호 제거
import re

def text_clean(doc):
    text =re.sub("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
    return text

In [10]:
# 경고 문구 삭제
import warnings
warnings.filterwarnings(action='ignore')

In [11]:
sql = "SET @@global.sql_mode= '';"
cursor.execute(sql)

0

# 크롤링 데이터 db 자동 저장

In [ ]:
# 가져올 파일 리스트
files = ['글로벌(2017년)','글로벌(2018년)','글로벌(2019년)','글로벌(2020년)',
        '금융(2017년)','금융(2018년)','금융(2019년)','금융(2020년)',
        '증권(2017년)','증권(2018년)','증권(2020년)']

In [12]:
import timeit # 작업시간 확인용

start_time = timeit.default_timer() # 시작 시간 체크

for fl in files:
  
    temp_df = pd.read_csv('./전처리 데이터/result(naver_news)-{}.csv'.format(fl))
    temp_df.drop('Unnamed: 0',axis=1,inplace=True)
    temp_df = temp_df.fillna('0') # 제목 공백 데이터 임의값 처리.. drop처리시 개수 매칭 오류 
    print('{} 데이터 작업중..'.format(fl))

    # 데이터 섹터 구분 칼럼 추가 (DB에서 작성글 분류 필요시 하기 위함)
    
    if '글로벌' in fl :
        temp_df['분류'] = '글로벌'

    elif '금융' in fl :
        temp_df['분류'] = '금융'
        
    elif '증권' in fl :
        temp_df['분류'] = '증권'
    
        
    # 특수기호 제거 함수식 데이터프레임 적용
    for i in range(len(temp_df)):
        
        # 불필요한 크롤링 코드가 제목에 들어가 있어서 길면 예외처리
        if len(temp_df['제목'][i]) > 150 : # 150자 이상 제목 거의 없음
            pass
        
        # 이메일 형태 제거 -> 이메일이 없는 기사가 있어 예외처리
        try: # 이메일 형태가 있을 때
            temp_df['본문'][i] = re.sub('([a-zA-Z0-9_.+m]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)','',temp_df['본문'][i])
            temp_df['본문'][i] = re.sub('([a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)','',temp_df['본문'][i])
            temp_df['제목'][i] = text_clean(temp_df['제목'][i])
            temp_df['본문'][i] = text_clean(temp_df['본문'][i])
            
        except: # 이메일 형태가 없을 때
            try:
                temp_df['제목'][i] = text_clean(temp_df['제목'][i])
                temp_df['본문'][i] = text_clean(temp_df['본문'][i])
            except:
                try: # 제목은 비어있는데 본문은 살아있는 경우
                    temp_df['본문'][i] = text_clean(temp_df['본문'][i])
                except:
                    pass
                
    # 데이터 db 저장
    for i in range(len(temp_df)):

        v1 = temp_df.iloc[i][0]
        v2 = temp_df.iloc[i][1]
        v3 = temp_df.iloc[i][2]
        v4 = temp_df.iloc[i][3]

        sql = """insert into news_table values ('{}','{}','{}','{}');""".format(v1,v2,v3,v4)

        cursor.execute(sql)
    
    terminate_time = timeit.default_timer() # 종료 시간 체크   
    print("현재 작업까지 총 %f초 걸렸습니다." % (terminate_time - start_time))
    print()
print('작업이 완료되었습니다.')

금융(2017년) 데이터 작업중..
현재 작업까지 총 1031.017671초 걸렸습니다.

증권(2017년) 데이터 작업중..
현재 작업까지 총 7153.915990초 걸렸습니다.

증권(2018년) 데이터 작업중..
현재 작업까지 총 17314.877625초 걸렸습니다.

증권(2019년) 데이터 작업중..
현재 작업까지 총 26302.467671초 걸렸습니다.

증권(2020년) 데이터 작업중..
현재 작업까지 총 33962.666028초 걸렸습니다.



In [20]:
# db 반영 개수 확인
sql = "SELECT COUNT(제목) FROM news_table where 분류 = '금융';"
cursor.execute(sql)
cursor.fetchall()

((486083,),)

In [13]:
# 입력 내용 db 반영
sql = 'commit;'
cursor.execute(sql)
cursor.fetchall()

()